# Spam-Filter mit Bayes

Die [Text REtrieval Conference (TREC) Spam Track Konferenz](https://trec.nist.gov/pubs/trec16/papers/SPAM.OVERVIEW16.pdf) im Jahr 2007 hatte das Ziel die Erkennung vom SPAM-Nachrichten zu verbessern.

Um die Methoden zu evaluieren wurde ein [Datensatz](http://plg.uwaterloo.ca/~gvcormac/treccorpus07/) aus 75419 E-Mails verwendet. Die Daten bestehen aus 25220 Nicht-SPAM-Nachrichten und 50199 SPAM-Nachrichten. Die E-Mails wurden dabei zwischen dem 8. April und 6 Juli 2007 auf einem E-Mail-Server gesammelt.

Wir wollen einen Filter bauen, der eine Email an Hand Ihres Betreffs und Korpus als *Spam* oder *kein-Spam* klassifiziert.

In [1]:
# wir laden die Daten
import email

nospam, spam = [], []
with open('../Daten/trec07p/full/index','r') as fin:
    for line in fin:
        classification, fn = line.strip().split(' ')
        with open('../Daten/trec07p/full/'+fn, errors='ignore') as fin:
            if classification == 'ham':
                nospam.append(email.message_from_file(fin))
            elif classification == 'spam':
                spam.append(email.message_from_file(fin))
            else:
                raise 'ERROR: Unknown Classifikation: %s' % classification
            
print('Anzahl der Spam-Mails: %d' % len(spam))
print('Anzahl der No-Spam-Mails: %d' % len(nospam))

Anzahl der Spam-Mails: 50199
Anzahl der No-Spam-Mails: 25220


#### Als nächstes werden aus den Betreffs und E-Mail-Texten Wortlisten, wobei Sonderzeichen und Zahlen ignoriert werden.

In [2]:
import nltk
import re

stemmer = nltk.PorterStemmer()
stopwords = ('and', 'further', 'he', 'with', 'those', 're', 'to', "she's", 'same', 'this', 'only', 'its', 'but', 'both', 'him', 'too', 'himself', 'them', 'any', "you'll", 'from', 'here', "don't", 'we', 'you', 'yourselves', 'between', 'been', "won't", 'after', 'theirs', 'most', 'shan', 'i', 'nor', 'below', 'do', 'd', 'should', 'which', 'under', 'her', 'am', "needn't", "wouldn't", 'their', 'what', 'have', 'me', 'during', 'just', 'is', 'where', "wasn't", 'when', 'all', 'themselves', 'few', 'now', 't', 'in', "couldn't", "hasn't", 'ma', 'into', 'she', 'other', 'my', 'did', 'll', 'no', 'wouldn', 'some', 'such', 'out', 'aren', 'a', "didn't", 'ours', 'having', 'an', 'shouldn', 'hasn', 'off', 'didn', 'against', 'ourselves', 'doing', 'again', 'ain', 'the', 'if', 'were', 'as', "mustn't", 'at', 'by', 'couldn', 'itself', 'your', 'before', 'each', "you're", 'has', "isn't", "shan't", 'they', 'herself', 'y', "haven't", 'needn', 'his', 'because', 'own', "you've", 's', "you'd", "that'll", "aren't", "weren't", 'more', 'whom', 'being', "doesn't", 'then', 'does', 'so', 'while', 'why', 'who', 'once', 'will', 'isn', 'weren', "should've", 'mustn', 'won', 'be', 'hers', 'had', "shouldn't", 'about', 'or', 'above', 'there', 'o', 'doesn', "mightn't", 'over', "hadn't", 'wasn', 'was', 'hadn', 'up', 'yourself', 've', 'our', 'can', 'on', 'm', 'for', 'very', 'myself', 'of', 'don', 'haven', 'how', 'yours', 'through', 'until', 'these', 'that', "it's", 'are', 'than', 'it', 'mightn', 'not', 'down')

def nachr2str(parts):
    ret = ''
    if type(parts) == str:
        ret += " " + parts
    elif type(parts) == list:
        for part in parts:
            ret += nachr2str(part.get_payload())
    elif parts.get_content_type == 'text/plain':
        ret += parts.get_payload()
    return ret

def extrahiere_woerter(mail):
    betreff = mail['Subject']
    if not betreff:
        betreff = ''
    text = betreff + ' ' + nachr2str(mail.get_payload())
    # extrahiere Wörter, mach aus einem Betreff und Text eine Wortliste aus Kleinbuchstachen ohne Duplikate
    tokens = set(re.findall('[^\W\d]+', text.lower()))

    # bilde den Wortstamm und entferne Stoppwörter
    return set([stemmer.stem(w) for w in tokens if w not in stopwords])

spam_wortlisten = []
nospam_wortlisten = []
for mail in nospam:
    nospam_wortlisten.append(extrahiere_woerter(mail))
for mail in spam:
    spam_wortlisten.append(extrahiere_woerter(mail))

In [3]:
print(spam_wortlisten[0])
print(nospam_wortlisten[0])

{'gr', 'margin', 'com', 'bodi', 'ffff', 'px', 'tr', 'pressur', 'v', 'b', 'align', 'html', 'past', 'excoriationtuh', 'bottom', 'border', 'width', 'ffffff', 'back', 'rise', 'bgcolor', 'cellspac', 'color', 'ccffaa', 'brand', 'center', 'ciali', 'qualiti', 'tri', 'td', 'br', 'feel', 'http', 'gener', 'div', 'right', 'self', 'solid', 'tabl', 'style', 'span', 'anxieti', 'cellpad', 'ia', 'href', 'occas', 'perform', 'old', 'thing', 'lzmfnrdklek'}
{'pass', 'com', 'seem', 'fr', 'listmast', 'unstabl', 'autom', 'org', 'releas', 'like', 'replac', 'list', 'html', 'en', 'hi', 'etch', 'readm', 'littl', 'libr', 'unsubscrib', 'name', 'updat', 'savoirfairelinux', 'lenni', 'logiciel', 'contact', 'develop', 'mirror', 'yan', 'propog', 'ftp', 'gulu', 'subject', 'http', 'dist', 'consult', 'ca', 'request', 'troubl', 'usherbrook', 'current', 'exampl', 'typo', 'email', 'file', 'test', 'debian', 'check', 'packag', 'snapshot', 'morin', 'access'}


#### 70% der Daten sollen für das Training und die anderen für den Test benutzt werden.

In [4]:
import random

random.seed(4711)

# verwürfle die Listen
random.shuffle(spam_wortlisten)
random.shuffle(nospam_wortlisten)

# 70% der Daten sollen zum Training die anderen 30% zum Testen genutzt werden
m = 70
train_spam, test_spam = spam_wortlisten[:m*len(spam_wortlisten)//100], spam_wortlisten[m*len(spam_wortlisten)//100:]
train_nospam, test_nospam = nospam_wortlisten[:m*len(nospam_wortlisten)//100], nospam_wortlisten[m*len(nospam_wortlisten)//100:]

#### Wir bauen nun den Bayes-Klassifizierer

In [5]:
from collections import defaultdict

N_Spam = len(train_spam)
N_NoSpam = len(train_nospam)

wortliste_spam, wortliste_nospam = defaultdict(int), defaultdict(int)

for wortliste in train_spam:
    for wort in wortliste:
        wortliste_spam[wort] += 1
        
for wortliste in train_nospam:
    for wort in wortliste:
        wortliste_nospam[wort] += 1

### Der Klassifizierer

Jetzt haben wir die Häufigkeiten/Wahrscheinlichkeiten, um unbekannte Nachrichten zu klassifizieren:

$$ \frac{P(\text{Spam} \;|\; w_1 \land \ldots \land w_n)}{P(\overline{\text{Spam}} \;|\; w_1 \land \ldots \land w_n)}  = \left( \prod_{i=1}^n \frac{N_{\text{Spam}, w_i}+1}{N_{\text{Spam}}+2}\right) \cdot \left( \prod_{i=1}^n \frac{N_{\overline{\text{Spam}}, w_i}+1}{N_{\overline{\text{Spam}}}+2}\right)^{-1} \cdot \frac{N_{\text{Spam}}+2}{N_{\overline{\text{Spam}}}+2} $$

In [6]:
from functools import reduce
from operator import mul

def klassifiziereBetreff(wortliste):
    return ((N_Spam+2)/(N_NoSpam+2)) * reduce(mul, [((wortliste_spam[wort]+1)/(N_Spam+2)) / ((wortliste_nospam[wort]+1)/(N_NoSpam+2)) for wort in wortliste], 1)

schwelle = 1  # Klassifizierungsschwellwert

korrekt_spam = fehler_spam = 0
for wortliste in test_spam:
    if klassifiziereBetreff(wortliste) >= schwelle:
        korrekt_spam += 1
    else:
        fehler_spam += 1

korrekt_nospam = fehler_nospam = 0
for wortliste in test_nospam:
    if klassifiziereBetreff(wortliste) >= schwelle:
        fehler_nospam += 1
    else:
        korrekt_nospam += 1

print('           erkannt (%)    nicht erkannt (%)')
print('-'*43)
print('Spam:       {0:4d} {1:5.01f}           {2:4d} {3:5.01f}'.format(korrekt_spam, korrekt_spam*100/len(test_spam), fehler_spam, fehler_spam*100/len(test_spam)))
print('No-Spam:    {0:4d} {1:5.01f}           {2:4d} {3:5.01f}'.format(korrekt_nospam, korrekt_nospam*100/len(test_nospam), fehler_nospam, fehler_nospam*100/len(test_nospam)))
print('Insgesamt:  {0:4d} {1:5.01f}           {2:4d} {3:5.01f}'.format(korrekt_spam+korrekt_nospam, (korrekt_spam+korrekt_nospam)*100/(len(test_spam)+len(test_nospam)), fehler_spam+fehler_nospam, (fehler_spam+fehler_nospam)*100/(len(test_spam)+len(test_nospam))))


           erkannt (%)    nicht erkannt (%)
-------------------------------------------
Spam:       14449  95.9            611   4.1
No-Spam:    7511  99.3             55   0.7
Insgesamt:  21960  97.1            666   2.9
